In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***Multinomial Logistic Regression***
Multiclass Classification Problem

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,accuracy_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
data = pd.read_csv('/content/drive/MyDrive/KP_Module (1)/Practical Machine Learning/Practise/Cases/Image Segmentation/Image_Segmention.csv')

In [6]:
# Since our Data is not divided by the classes so we need to specify the classes
# thats why we are using One-Hot LabelEncoder on Class column
#initiating the Label Encoder
lbl = LabelEncoder()
data['Class']=lbl.fit_transform(data['Class'])

X = data.drop('Class',axis=1)
y = data.Class

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=23,stratify=y)

In [10]:
#Intantiating the Model
lr = LogisticRegression(multi_class='multinomial')

#fiting the Model
lr.fit(X_train,y_train)

#making the Probability Prediction on Test Data
y_pred_proba = lr.predict_proba(X_test)
y_pred = lr.predict(X_test)

#Calculatin the loss using LogLoss Functions
print("The Error using LogLoss Function",log_loss(y_test,y_pred_proba))
# Logloss is used beacuse the predictions are in 0 or 1 beacause it a classification problem

#checking the Accuracy
print("The Accuracy of the Logistic Regression Model is :",accuracy_score(y_test,y_pred))

0.27163435436137845
0.9206349206349206


#**Performing GridSearchCV**

In [16]:

# Tunning the penalty and solver paramters
from sklearn.model_selection import GridSearchCV,StratifiedKFold

# penalties=['l1', 'l2', 'ElasticNet',None]
# params={'penalty':penalties}


params={'penalty':['l1','l2','ElasticNet',None],
        'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
        'multi_class':['ovr','multinomial']}
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=23)


gcv = GridSearchCV(lr,param_grid=params,cv=kfold)

gcv.fit(X,y)
#print(gcv.cv_results_)
print("Best Parameter:",gcv.best_params_)
print("Best Result:", gcv.best_score_)


#for LogLoss
gcv = GridSearchCV(lr,param_grid=params,cv=kfold,scoring='neg_log_loss')

gcv.fit(X,y)
#print(gcv.cv_results_)
print("For LogLoss")
print("Best Parameter:",gcv.best_params_)
print("Best Result:", gcv.best_score_)



Best Parameter: {'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'newton-cg'}
Best Result: 0.9044134727061557
For LogLoss
Best Parameter: {'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'newton-cg'}
Best Result: -0.541891920389238
Best Model Result :  1.0


#Inferencing

In [19]:

#Best Model Result
best_model = gcv.best_estimator_
print("Coefficients :",best_model.coef_)
print("Intercept :",best_model.intercept_)

#loading the unlabeled Data
test_data = pd.read_csv("/content/drive/MyDrive/KP_Module (1)/Practical Machine Learning/Practise/Cases/Image Segmentation/tst_img.csv")

#test_pred_proba = best_model.predict_proba(test_data)
#np.argmax(test_pred_proba,axis=1)
#or

test_pred = best_model.predict(test_data)
print(lbl.inverse_transform(test_pred))

Coefficients : [[-3.03795362e-02 -1.30221520e-01  1.09226712e-04 -1.64184590e-02
  -4.54557132e-05 -2.65724870e-01 -1.25079732e-01  3.68951411e-02
  -9.54638292e-02 -7.54355828e-02  2.68940447e-01 -3.36888680e-02
  -4.61558893e-01  1.03312900e+00  1.25240577e-01 -1.15836925e+00
  -5.48327806e-02 -2.49587040e-02  7.20254093e-02]
 [-5.98940732e-03 -6.07565221e-02  2.25631395e-04  1.13998006e-01
   7.06925000e-02 -8.39640647e-02  1.49657538e-01  2.94563862e-01
  -7.08050377e-02  1.37768072e-01  2.77939178e-01  2.32388941e-01
  -9.70245851e-02  4.20511012e-01  2.83864994e-01 -7.04376861e-01
  -5.77184923e-01  2.11651664e-02 -1.79073709e-01]
 [-9.11296986e-03 -1.70647342e-01 -7.29667010e-04 -1.92285867e-02
   1.27829379e-03  2.86603812e-01  6.25184418e-02 -5.10365021e-02
   4.79597499e-02 -2.75403829e-01 -5.34961732e-01 -3.47057446e-01
   5.58066421e-02 -7.78673875e-01 -2.14958328e-01  9.93632295e-01
   7.12168523e-01  7.45075538e-01 -5.94208445e-01]
 [ 1.84401544e-02  2.22727912e-01 -7.601